In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/EFQM/session 5/LSTM-Multivariate_pollution.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour


data.drop(['date'], axis=1, inplace=True)
data.head()

In [ ]:
dummy_wind = pd.get_dummies(data['wnd_dir'])
data = pd.concat([data, dummy_wind], axis=1)
data.drop(['wnd_dir'], axis=1, inplace=True)
data.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = data.drop(['pollution'], axis=1)
y = data['pollution']

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X)

scaled_data = pd.DataFrame(scaled_data)
scaled_data['pollution'] = y
scaled_data.head()

In [ ]:
import numpy as np

X = []
y = []

sequence_length = 16
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data.iloc[i-sequence_length:i].values)
    y.append(scaled_data.iloc[i]['pollution'])

X, y = np.array(X), np.array(y)
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])), # to ignore padding
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, epochs=32, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

In [ ]:
model.summary()